# **CommonLit Readability Assessment**

## Determinining Baseline Performance with 1D ConvNet with Pre-trained Word Embeddings

In [ ]:
# Imports required packages

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Some initialization

sns.set_theme()

### Loading Data & Counting Samples

In [ ]:
# Loads data

train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
submission = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")

In [ ]:
# Checks how the data looks from top few samples

display(train.head())

## Data Preparation

In [ ]:
# Making all excerpts in lowercase

train.excerpt = train.excerpt.str.lower()

### Segmenting (Discrete Interval) Label

As the distribution of labels are not equal, the labels should be splitted with stratification so that each fold get nearly equal distribution from data segments. 

In [ ]:
# Sets configurations

n_bins = 20
n_splits = 5

In [ ]:
# Segments discrete interval of label by marking each sample with a bin number

train["bin"] = pd.cut(x=train.target, bins=n_bins, labels=[i for i in range(n_bins)])

In [ ]:
# Ensurs interval of each bin is near to equal

display(train.groupby(["bin"]).target.apply(lambda x: x.max()-x.min()))

In [ ]:
# Creates K-Fold cross validator

k_fold_cv= StratifiedKFold(n_splits=n_splits, shuffle=True)

validation_scores = []    # Stores validation scores across folds
test_predictions = np.zeros(shape=(len(submission), n_splits))

In [ ]:
# Create K-Fold cross validation splitter

k_fold_cv_generator = k_fold_cv.split(train, y=train.bin)

# Plots to verify if every split gets equal distribution of samples from every bin

plt.figure(figsize=(12, 6))

for fold, (id_x_train, id_x_val) in enumerate(k_fold_cv_generator):
    sns.histplot(data=train.iloc[id_x_train], x="target", bins=n_bins, fill=False, label=f"Fold {fold}")

plt.title('Label Distribution for Each Fold')
plt.legend()
plt.show()

In [ ]:
# Removes columns considered not to be required

train.drop(["url_legal", "license"], axis=1, inplace=True)
test.drop(["url_legal", "license"], axis=1, inplace=True)

## Configurations

In [ ]:
max_features = 30000    # Vocabulary size was little more than 30000 found during analysis
max_length = 250        # Considering a little more than 205 found during analysis
embedding_dim = 300     # 300 dimensional GloVe word embedding is being considered
batch_size = 32         # Size of the batch of samples to take as input at a time during model training
epochs = 20             # Number of epochs to train model

## Modeling

In [ ]:
# Creates vectorizer
vectorizer = TextVectorization(max_tokens=max_features, output_sequence_length=max_length)

In [ ]:
# Extracts and loads vectors from GloVe word embeddings

embeddings_index = {}

with open("../input/glove6b/glove.6B.300d.txt") as file:
    for i, line in enumerate(file):
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, dtype=float, sep=" ")
        embeddings_index[word] = coefs

In [ ]:
fig, axes = plt.subplots(1, n_splits, sharex=True, sharey=True, figsize=(20, 5))
fig.suptitle("K-Fold Cross Validation Performance")

k_fold_cv_generator = k_fold_cv.split(train, y=train.bin)

# Iterates over each fold to train the linear regression model, 
# predict on validation data and stores into a list to get its mean at the end

for fold, (idx_train, idx_val) in enumerate(k_fold_cv_generator):
    print(f"Fold {fold}...")

    text_ds = tf.data.Dataset.from_tensor_slices(train.excerpt.iloc[idx_train]).batch(32)
    vectorizer.adapt(text_ds)
    
    voc = vectorizer.get_vocabulary()
    word_index = dict(zip(voc, range(len(voc))))
    
    missing_words_in_pretrained_embeddings = 0
    embeddings_matrix = np.zeros((max_features, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector
        else:
            missing_words_in_pretrained_embeddings += 1
    print(f"\tNumber of missing words in Glove: {missing_words_in_pretrained_embeddings}")
    
    embedding_layer = layers.Embedding(max_features, 
                                   embedding_dim, 
                                   embeddings_initializer=keras.initializers.Constant(embeddings_matrix),
                                   input_length=max_length, 
                                   trainable=False)
    
    sequences_input = layers.Input(shape=(None,), dtype="int64")
    x = embedding_layer(sequences_input)
    x = layers.Conv1D(128, 5, activation="relu")(x)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(128, 5, activation="relu")(x)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(128, 5, activation="relu")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(128, activation="relu")(x)
    #x = layers.Dropout(0.1)(x)
    predictions_output = layers.Dense(1)(x)

    model = keras.Model(sequences_input, predictions_output)
    
    x_train = vectorizer(np.array([[s] for s in train.excerpt.iloc[idx_train]])).numpy()
    x_val = vectorizer(np.array([[s] for s in train.excerpt.iloc[idx_val]])).numpy()

    y_train = np.array(train.target.iloc[idx_train])
    y_val = np.array(train.target.iloc[idx_val])
    
    print("\tTraining model...", end=" ")
    model.compile(optimizer="rmsprop", loss= "mse", metrics=[RootMeanSquaredError()])
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, 
                        verbose=False, shuffle=False, validation_data=(x_val, y_val))
    print("completed.")
    
    history = history.history
    
    axes[fold].set_title(f"Fold {fold}")
    axes[fold].plot(range(1, len(history["root_mean_squared_error"]) + 1), history["root_mean_squared_error"], "bo", label="Loss")
    axes[fold].plot(range(1, len(history["val_root_mean_squared_error"]) + 1), history["val_root_mean_squared_error"], "b", label="Validation Loss")
    axes[fold].set_xlabel("Epochs")
    if fold == 0:
        axes[fold].set_ylabel("Loss (RMSE)")
    axes[fold].legend()
    
    rmse = np.mean(history["val_root_mean_squared_error"])
    validation_scores.append(rmse)
    print(f"\tRMSE over validation data: {rmse}")
    # Performs predictions on test data
    print(f"\tPerforming predictions on fold {fold}...", end=" ")
    x_test = vectorizer(np.array([[s] for s in test.excerpt])).numpy()
    predictions = model.predict(x_test)
    predictions.resize((len(test)))
    test_predictions[:,fold] = predictions.copy()
    print("done.")
    print()
    
print(f"K-Fold cross validation RMSE performance: {np.mean(validation_scores)}")

## Submission

In [ ]:
# Averaging predictions across folds

submission.target = test_predictions.mean(axis=1)

In [ ]:
# Submitting by saving predictions into submission file

submission.to_csv("submission.csv", index=False)